In [149]:
# this notebook was used in the initial development of the recommender system, and for testing code before full integration
# it does not contain all of the functionality present in the main application.
# It was included in the submission as it provides a clearer step by step implementation
# of the basic recommender system that was implemented into the application.

from sklearn.preprocessing import MinMaxScaler
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from sklearn.feature_extraction.text import TfidfVectorizer

scope='user-library-read playlist-modify-public playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[2]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []
genres_list = []

for track in playlist['items']:
    
    track_data = {
            'danceability': None,
            'energy': None,
            'loudness': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None,
            'genres': None,
            'track_pop': None,
            'id': track['track']['id']
        }
    
    def get_genres(track_id):
        # Get track information
        track_info = sp.track(track_id)
            
        # Get the list of genres for the first artist of the track
        if 'artists' in track_info and track_info['artists']:
            artist_id = track_info['artists'][0]['id']
            artist_info = sp.artist(artist_id)
                
            if 'genres' in artist_info:
                return artist_info['genres']
            
        return None
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['loudness'] = audio_features['loudness']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        track_id = track['track']['id']
        genres = get_genres(track_id)
        for genre in genres:
            genres_list.append(genre)
        track_info = sp.track(track_id)
        popularity = track_info['popularity']
        track_data['track_pop'] = popularity
        
        tracks_data.append(track_data)

playlist_features = pd.DataFrame(tracks_data)
playlist_features.drop_duplicates('id')

columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_pop']
data_to_scale = playlist_features[columns_to_scale]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)

# Replace the original values with the scaled values in the DataFrame
playlist_features[columns_to_scale] = scaled_data

genres_list = list(set(genres_list))
print(genres_list)
playlist_features.head()


['hip hop', 'atl hip hop', 'conscious hip hop', 'rap', 'west coast rap', 'chicago rap']


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres,track_pop,id
0,0.589831,0.263323,0.486808,0.188927,0.303448,0.000,0.544929,0.411215,0.000000,None,0.000000,0UtnpKaReKUg2GquaSxCyD
1,0.000000,0.059561,0.000000,0.000000,1.000000,0.356,0.163132,0.000000,1.000000,None,1.000000,7rbECVPkY5UODxoOUVKZnA
2,0.861017,1.000000,1.000000,0.752307,0.341872,0.000,0.000000,0.811215,0.318617,None,0.666667,3iVcZ5G6tvkXZkZKlMpIUs
3,1.000000,0.592476,0.379784,0.014085,0.000000,1.000,1.000000,0.280374,0.619628,None,0.733333,2t8yVaLvJ0RenpXUIAC52d
4,0.413559,0.000000,0.517651,1.000000,0.714286,0.000,0.032009,1.000000,0.093340,None,0.533333,5cc9Zbfp9u10sfJeKZ3h16


In [150]:
import ast
import pandas as pd
import os

from sklearn.preprocessing import MinMaxScaler

current_dir = os.getcwd()
relative_path = '../data/tracksgenres.csv'
file_path = os.path.join(current_dir, relative_path)

# Read the CSV file using pandas read_csv
all_songs_df = pd.read_csv(file_path)

#all_songs_df = pd.read_csv(r'C:\Users\alext\OneDrive\Desktop\ml_data\tracksgenres.csv')

#rearrange columns to match the user dataframe
all_songs_df_order = ['artists','genres', 'id', 'track_pop','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
all_songs_features_order = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo', 'track_pop', 'id']
all_songs_df = all_songs_df[all_songs_df_order]
all_songs_features = all_songs_df[all_songs_features_order]

columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_pop']
data_to_scale = all_songs_features[columns_to_scale]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
all_songs_features[columns_to_scale] = scaled_data


all_songs_features.head()


C:\Users\alext\AppData\Local\Temp\ipykernel_14396\815815102.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_songs_features[columns_to_scale] = scaled_data


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_pop,id
0,0.706357,0.219,0.735805,0.436533,0.995984,0.000000,0.178,0.539,0.430683,0.050505,1YZkudiD6LfY5vmZKdOTyO
1,0.442987,0.311,0.701068,0.337461,0.963855,0.243000,0.211,0.392,0.313490,0.060606,36OG5Ep6IPA4Mj6TxoTpcw
2,0.360242,0.185,0.708134,0.031889,0.805221,0.000000,0.105,0.303,0.444531,0.151515,0JqKK01MXbnDhBKKdcUCnX
3,0.581231,0.299,0.756929,0.068215,0.881526,0.000009,0.409,0.983,0.817413,0.080808,2iKCL9YGGTpDPen0LhzcX2
4,0.254289,0.234,0.755400,0.031269,0.878514,0.000000,0.371,0.295,0.388277,0.010101,2fwms7R3b5N8aBnJ1CaltH


In [151]:
"""
non_empty_genres_df = all_songs_features[all_songs_features['genres'].apply(len) > 0]

# Specify the maximum number of features
max_features = 500  # You can adjust this number based on your memory constraints

# Create the TfidfVectorizer with max_features
tfidf = TfidfVectorizer(max_features=max_features)

# Apply TF-IDF vectorization to non-empty genres
tfidf_matrix = tfidf.fit_transform(non_empty_genres_df['genres'].apply(lambda x: " ".join(x)))

# Create DataFrame from the TF-IDF matrix
genre_df = pd.DataFrame(tfidf_matrix.toarray(), columns=['genre' + "|" + i for i in tfidf.get_feature_names_out()])

# Reset the index of the DataFrame
genre_df.reset_index(drop=True, inplace=True)
"""


'\nnon_empty_genres_df = all_songs_features[all_songs_features[\'genres\'].apply(len) > 0]\n\n# Specify the maximum number of features\nmax_features = 500  # You can adjust this number based on your memory constraints\n\n# Create the TfidfVectorizer with max_features\ntfidf = TfidfVectorizer(max_features=max_features)\n\n# Apply TF-IDF vectorization to non-empty genres\ntfidf_matrix = tfidf.fit_transform(non_empty_genres_df[\'genres\'].apply(lambda x: " ".join(x)))\n\n# Create DataFrame from the TF-IDF matrix\ngenre_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[\'genre\' + "|" + i for i in tfidf.get_feature_names_out()])\n\n# Reset the index of the DataFrame\ngenre_df.reset_index(drop=True, inplace=True)\n'

In [152]:
def generate_playlist_feature(complete_feature_set, playlist_df):

    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_playlist_output = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_output.sum(axis = 0)

In [153]:
playlistfeatures = generate_playlist_feature(playlist_features, all_songs_df)

i = 0
while i <= len(playlist_features):
    playlistfeatures[i] = playlistfeatures[i]/len(playlist_features)
    i+=1

playlistfeatures.pop('genres')
# "single song" matrix
print(playlistfeatures)

danceability        0.372203
energy              0.330408
loudness            0.275957
speechiness         0.153278
acousticness        0.268374
instrumentalness      0.2712
liveness            1.163132
valence             1.091589
tempo               1.938245
track_pop                2.4
dtype: object


In [154]:
#all_songs_df = all_songs_df[all_songs_df['genres'] != "[]"] 

all_songs_df['genres'] = all_songs_df['genres'].apply(ast.literal_eval)

#all_songs_df = all_songs_df[all_songs_df['genres'].apply(lambda x: any(genre in genres_list for genre in x))]

# Display the filtered DataFrame
print(all_songs_df)

                                      artists  \
0                               ['Guru Dutt']   
1                               ['Guru Dutt']   
2                   ['The De Castro Sisters']   
3                   ['The De Castro Sisters']   
4                   ['The De Castro Sisters']   
...                                       ...   
470033  ['Danny Davis & The Nashville Brass']   
470034                               ['Ph03']   
470035                       ['Pitchshifter']   
470036                               ['Ludo']   
470037                               ['Ludo']   

                                                   genres  \
0                                                      []   
1                                                      []   
2                                                      []   
3                                                      []   
4                                                      []   
...                                          

In [155]:
from sklearn.metrics.pairwise import cosine_similarity

def generate_playlist_recos(df, features, nonplaylist_features, genres):

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df = non_playlist_df[non_playlist_df['genres'].apply(lambda x: any(genre in genres for genre in x))]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [156]:
print(playlistfeatures)

danceability        0.372203
energy              0.330408
loudness            0.275957
speechiness         0.153278
acousticness        0.268374
instrumentalness      0.2712
liveness            1.163132
valence             1.091589
tempo               1.938245
track_pop                2.4
dtype: object


In [157]:
all_songs_features.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_pop,id
0,0.706357,0.219,0.735805,0.436533,0.995984,0.000000,0.178,0.539,0.430683,0.050505,1YZkudiD6LfY5vmZKdOTyO
1,0.442987,0.311,0.701068,0.337461,0.963855,0.243000,0.211,0.392,0.313490,0.060606,36OG5Ep6IPA4Mj6TxoTpcw
2,0.360242,0.185,0.708134,0.031889,0.805221,0.000000,0.105,0.303,0.444531,0.151515,0JqKK01MXbnDhBKKdcUCnX
3,0.581231,0.299,0.756929,0.068215,0.881526,0.000009,0.409,0.983,0.817413,0.080808,2iKCL9YGGTpDPen0LhzcX2
4,0.254289,0.234,0.755400,0.031269,0.878514,0.000000,0.371,0.295,0.388277,0.010101,2fwms7R3b5N8aBnJ1CaltH


In [158]:
recos = generate_playlist_recos(all_songs_df, playlistfeatures, all_songs_features, genres_list)

recos.head(20)



,artists,genres,id,track_pop,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,sim
404406,['Brent Faiyaz'],"[dmv rap, pop, r&b, rap]",5VlTQnZO89Ioku8ssdbqJk,83,0.505,0.267,-13.763,0.0500,0.279000,0.000000,0.0960,0.439,143.929,0.824407
403126,['Don Toliver'],"[rap, trap]",527k23H0A4Q0UJN3vGs0Da,84,0.629,0.692,-8.045,0.0376,0.009810,0.331000,0.6030,0.453,162.948,0.816670
435061,['DANGERDOOM'],"[alternative hip hop, hip hop]",5ddNrCDGZyV0vM0Q2dvePC,56,0.427,0.525,-8.035,0.2050,0.140000,0.000073,0.6020,0.578,171.793,0.806979
358722,['LL Cool J'],"[east coast hip hop, hardcore hip hop, hip hop...",6jL1SnyXcXiKOmw4M2RnmT,56,0.620,0.445,-17.154,0.3820,0.002730,0.014500,0.3330,0.587,202.134,0.801890
405690,['Childish Gambino'],"[atl hip hop, hip hop, pop rap, rap]",0wXuerDYiBnERgIpbb3JBR,84,0.743,0.347,-11.174,0.1210,0.167000,0.009510,0.1030,0.572,160.143,0.799003
447828,['Lil Uzi Vert'],"[melodic rap, philly rap, rap, trap]",1gGsNNUfvVMOv4llVMg8Vg,70,0.522,0.428,-11.332,0.1430,0.235000,0.000002,0.2780,0.399,129.829,0.798479
358709,['LL Cool J'],"[east coast hip hop, hardcore hip hop, hip hop...",0hioWv2FDtVePjHFTwgrf7,54,0.624,0.435,-15.440,0.2450,0.000917,0.000001,0.4100,0.735,179.267,0.798251
405780,['Bryson Tiller'],"[kentucky hip hop, pop, pop rap, r&b, rap, trap]",43PuMrRfbyyuz4QpZ3oAwN,78,0.525,0.433,-10.598,0.1850,0.107000,0.000000,0.1350,0.276,160.108,0.797706
161846,['Juice WRLD'],"[chicago rap, melodic rap]",3XRQT7EoS4U87rUuJwg5P3,73,0.620,0.451,-7.623,0.1700,0.128000,0.000000,0.3730,0.294,156.906,0.796115
405686,['Childish Gambino'],"[atl hip hop, hip hop, pop rap, rap]",2ZltjIqztEpZtafc8w0I9t,78,0.472,0.448,-7.272,0.3260,0.113000,0.000000,0.0914,0.636,166.077,0.796054
